# Data Prepration

This notebook performs all data preparation steps required to construct the international citation preference networks used in the [paper](https://arxiv.org/abs/2404.05861). It starts from a local copy of the OpenAlex database that has been preprocessed using **pySciSci**.

Because this notebook relies on a locally stored OpenAlex snapshot, it will not run out of the box without access to these data. Users interested in reproducing the full data preparation pipeline should first follow the preprocessing steps described in the companion notebook:

**[Getting_Started_with_OpenAlex.ipynb](https://github.com/SciSciCollective/pyscisci/blob/master/examples/Getting_Started/Getting%20Started%20with%20OpenAlex.ipynb)**

The processed outputs generated here are archived in the shared data folder and are used as inputs to the analysis notebook, which reproduces all figures and regression results.

In [1]:

import pyscisci.all as pyscisci

import os
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pylab as plt

import nltk
#nltk.download('punkt')
import string
import re
import json
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline

In [2]:
# set this path to where the OpenAlex database is stored
path2openalex = '/Users/hgt6rn/Documents/DataSets/OpenAlex'

path2countrydata = "/Users/hgt6rn/Documents/DataSets/OpenAlex/precomputed_metrics"

In [3]:
myoa = pyscisci.OpenAlex(path2openalex, database_extension='csv.gz', keep_in_memory=False) 
# set keep_in_memory=False if you want to load the database each time its needed - good for when you 
# cant keep more than one dataframe in memory at a time
# otherwise keep_in_memory=True will keep each database in memory after its loaded

# Create the publication to country mapping

In [4]:
pub = myoa.load_publications(columns =['PublicationId', 'Year', 'DocType', 'JournalId'])
pub.shape

Loading Publications:   0%|          | 0/513 [00:00<?, ?it/s]

/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Colu

/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Colu

/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Colu

(244557590, 4)

In [6]:

pub2doctype = pub[pub['DocType']=='article']
pub2doctype.dropna(inplace=True)

print(pub2doctype['PublicationId'].nunique(), "Articles")
pub2doctype = pub2doctype[np.logical_and(pub2doctype['Year'] >= 1990, pub2doctype['Year'] <= 2023)]
print(pub2doctype['PublicationId'].nunique(), "Articles in timespan")


pub2year = {pid:y for pid, y in pub2doctype[['PublicationId', 'Year']].values}
print(len(pub2year))

pubarticles = np.sort(pub2doctype['PublicationId'].values)
print(pubarticles.shape)

(153910134, 4)
(129520147, 4)
(128228946, 4)


In [8]:
aff_df = myoa.load_affiliations(columns = ['AffiliationId', 'Country'])
aff_df.dropna(inplace=True)
aff_df.reset_index(drop=True, inplace=True)

focus_affs = np.sort(aff_df['AffiliationId'].values)

aff_df

Loading Affiliations:   0%|          | 0/153 [00:00<?, ?it/s]

,AffiliationId,Country
0,4210161801,GB
1,4210127386,US
2,4210157748,US
3,4210095232,IL
4,4210128266,BE
...,...,...
107667,4210124170,DK
107668,4210119575,ES
107669,4210158772,FI
107670,4210135183,CN


In [10]:
paa_df = myoa.load_publicationauthoraffiliation(columns=['PublicationId', 'AuthorId', 'AffiliationId'],
                                                dropna=['AuthorId', 'AffiliationId'],
                                                filter_dict = {'PublicationId':pubarticles,
                                                             'AffiliationId':focus_affs})
print(paa_df.shape)
paa_df.nunique()

Loading Publication Author Affiliation:   0%|          | 0/513 [00:00<?, ?it/s]

(227796792, 3)


PublicationId    63983125
AuthorId         60791118
AffiliationId       94432
dtype: int64

In [11]:
paa_df['Year'] = paa_df['PublicationId'].map(pub2year)
paa_df.dropna(inplace=True)

paa_df = paa_df.merge(aff_df, how='left', on='AffiliationId')
paa_df.dropna(inplace=True)
print(paa_df.shape)

(227796792, 4)


In [12]:
pub2country = paa_df[['PublicationId', 'Country']].drop_duplicates(subset=['PublicationId', 'Country'])
pub2country.nunique()

PublicationId    63983125
Country               223
dtype: int64

In [13]:
pub2country.to_csv(os.path.join(path2countrydata, "oa_pub2country.csv.gz"), compression='gzip', mode='w', 
                   header=True, index=False)

In [13]:
npubs = pub2country.groupby('Country')['PublicationId'].nunique().to_frame().reset_index()
npubs = npubs.rename(columns={'PublicationId':'NumberPublications'})

npubs.to_csv(os.path.join(path2countrydata, 'oa_countryproductivity.csv.gz'), 
                          compression='gzip', mode='w', header=True, index=False)

npubs

,Country,NumberPublications
0,AD,131
1,AE,81660
2,AF,2572
3,AG,4547
4,AL,9224
...,...,...
218,XK,6113
219,YE,8512
220,ZA,426873
221,ZM,11393


In [9]:
pub2country = pd.read_csv(os.path.join(path2countrydata, "oa_pub2country.csv.gz"))
                   
pub2country['Year'] = pub2country['PublicationId'].map(pub2year)

npubs_y = pub2country.groupby(['Country', 'Year'])['PublicationId'].nunique().to_frame().reset_index()
npubs_y = npubs_y.rename(columns={'PublicationId':'NumberPublications'})

npubs_y.to_csv(os.path.join(path2countrydata, 'oa_countryproductivity_yearly.csv.gz'), 
                          compression='gzip', mode='w', header=True, index=False)

npubs_y

,Country,Year,NumberPublications
0,AD,1990.0,1
1,AD,1993.0,2
2,AD,1998.0,1
3,AD,2003.0,1
4,AD,2006.0,4
...,...,...,...
16964,ZW,2019.0,1125
16965,ZW,2020.0,1449
16966,ZW,2021.0,1750
16967,ZW,2022.0,1307


# Create the yearly national citation distributions

In [4]:
pub2country = pd.read_csv(os.path.join(path2countrydata, "oa_pub2country.csv.gz"))

kept_pids = np.sort(pub2country['PublicationId'].unique())
#kept_pids = np.sort(paa_df['PublicationId'].unique())
print(kept_pids.shape)

(63983125,)


In [6]:
pub2year = pd.read_csv(os.path.join(path2countrydata, "oa_pub2year.csv.gz"))
#pub2year = {pid:y for pid,y in pub2year[['PublicationId', 'Year']].values}

In [10]:
# change the timeperiod to focus on

window_start = 1990
window_end = 2023

kept_pids = np.sort(pub2country[np.logical_and(pub2country['Year'] >= window_start, pub2country['Year'] <= window_end)]['PublicationId'].unique())
print(kept_pids.shape)
pub2year = {pid:y for pid,y in pub2year[['PublicationId', 'Year']].values}

(57558268,)


In [ ]:
# Here we use a citation window
citation_window = 5

pub2ref = []
for fname in os.listdir(os.path.join(path2openalex, 'pub2ref')):
    if '.csv.gz' in fname:
        df = pd.read_csv(os.path.join(path2openalex, 'pub2ref', fname))
        
        df = df[pyscisci.isin_sorted(df['CitingPublicationId'].values, kept_pids)]
        df = df[pyscisci.isin_sorted(df['CitedPublicationId'].values, kept_pids)]
        
        df['CitingYear'] = [pub2year.get(pid, None) for pid in df['CitingPublicationId'].values]
        df.dropna(inplace=True)
        
        df['CitedYear'] = [pub2year.get(pid, None) for pid in df['CitedPublicationId'].values]
        df.dropna(inplace=True)
        
        df = df.loc[df['CitingYear'] >= df['CitedYear']]
        df = df.loc[df['CitingYear'] <=  df['CitedYear'].values + citation_window]
        
        pub2ref.append(df[['CitingPublicationId', 'CitingYear', 'CitedPublicationId', 'CitedYear']])
pub2ref = pd.concat(pub2ref)
print(pub2ref.shape)
pub2ref.drop_duplicates(subset=['CitingPublicationId', 'CitedPublicationId'], inplace=True)
print(pub2ref.shape)
pub2ref.nunique()

In [12]:
# change the 2017 if you have a more recent snapshot of OpenAlex

pub2ref = pub2ref[pub2ref['CitedYear'] <= 2017].reset_index(drop=True)
del pub2ref['CitingYear']
pub2ref.nunique()

CitingPublicationId    41431724
CitedPublicationId     30556883
CitedYear                   118
dtype: int64

In [ ]:
pub2ref = pub2ref.merge(pub2country[['PublicationId', 'Country']], how='left', left_on = 'CitingPublicationId', right_on='PublicationId')
del pub2ref['PublicationId']
pub2ref.rename(columns={'Country':'CitingCountry'}, inplace=True)

pub2ref = pub2ref.merge(pub2country[['PublicationId', 'Country']], how='left', left_on = 'CitedPublicationId', right_on='PublicationId')
del pub2ref['PublicationId']
pub2ref.rename(columns={'Country':'CitedCountry'}, inplace=True)

pub2ref.nunique()

In [13]:
ncites = pub2ref.groupby('CitedPublicationId', as_index=False)['CitingPublicationId'].nunique()
ncites = ncites.rename(columns={'CitingPublicationId':'C5'})
ncites = ncites.merge(pub2country[['PublicationId', 'Country']], how='left', left_on = 'CitedPublicationId', right_on='PublicationId')
del ncites['PublicationId']
ncites['CitedYear'] = ncites['CitedPublicationId'].map(pub2year)
ncites.rename(columns={'Country':'CitedCountry'}, inplace=True)

ncites.to_csv(os.path.join(path2countrydata, "oa_c5.csv.gz"), compression='gzip', mode='w', header=True, index=False)

In [20]:
countrycites = pub2ref.groupby(['CitedPublicationId', 'CitingCountry', 'CitedYear'], as_index=False)['CitingPublicationId'].nunique()

countrycites = countrycites.merge(pub2country, how='left', left_on='CitedPublicationId', right_on='PublicationId')
del countrycites['PublicationId']
countrycites = countrycites.rename(columns={'CitingPublicationId':'CountryCitations'})
print(countrycites.shape)
countrycites.to_csv(os.path.join(path2countrydata, "oa_countrycites.csv.gz"), compression='gzip', mode='w', header=True, index=False)



(242110245, 5)


# remove self-citations and same-organization citations

In [20]:

pub2authors = defaultdict(set)
pub2aff = defaultdict(set)
for pid, aid, aff in paa_df[['PublicationId', 'AuthorId', 'AffiliationId']].values:
    pub2authors[pid].add(aid)
    pub2aff[pid].add(aff)
    
print(len(pub2authors), len(pub2aff))

65018678 65018678


In [ ]:
noselfcite = np.array([len(pub2authors[citingpid] & pub2authors[citedpid]) == 0 for citingpid, citedpid in pub2ref[['CitingPublicationId', 'CitedPublicationId']].values])
print(np.count_nonzero(noselfcite==0))


countrycites = pub2ref.loc[noselfcite].groupby(['CitedPublicationId', 'CitingCountry', 'CitedYear'], as_index=False)['CitingPublicationId'].nunique()
countrycites = countrycites.merge(pub2country, how='left', left_on='CitedPublicationId', right_on='PublicationId')
del countrycites['PublicationId']
countrycites = countrycites.rename(columns={'CitingPublicationId':'CountryCitations'})

countrycites.to_csv(os.path.join(path2countrydata, "oa_countrycites_noself.csv.gz"), compression='gzip', mode='w', header=True, index=False)


In [25]:
notsameorg = np.array([len(pub2aff[citingpid] & pub2aff[citedpid]) == 0 for citingpid, citedpid in pub2ref[['CitingPublicationId', 'CitedPublicationId']].values])
print(np.count_nonzero(notsameorg==0))
notsameorg = np.logical_and(notsameorg, noselfcite)
print(np.count_nonzero(notsameorg==0))

countrycites = pub2ref.loc[notsameorg].groupby(['CitedPublicationId', 'CitingCountry', 'CitedYear'], as_index=False)['CitingPublicationId'].nunique()
countrycites = countrycites.merge(pub2country, how='left', left_on='CitedPublicationId', right_on='PublicationId')
del countrycites['PublicationId']
countrycites = countrycites.rename(columns={'CitingPublicationId':'CountryCitations'})

countrycites.to_csv(os.path.join(path2countrydata, "oa_countrycites_nosameorg.csv.gz"), compression='gzip', mode='w', header=True, index=False)

100036381
111793438


In [ ]:
# add the Cited Journal for bootsrap networks
countrycites = countrycites.merge(pub[['PublicationId', 'JournalId']], how='left', left_on='CitedPublicationId', right_on='PublicationId')
del countrycites['PublicationId']

countrycites = countrycites.rename(columns={'JournalId':'CitedJournalId'})

countrycites.to_csv(os.path.join(path2countrydata, "oa_countrycites_nosameorg_journal.csv.gz"), compression='gzip', mode='w', header=True, index=False)

# now get the collaboration networks

In [6]:
pub2country = pd.read_csv(os.path.join(path2countrydata, "oa_pub2country.csv.gz"))

In [27]:
pub2country['Year'] = pub2country['PublicationId'].map(pub2year)

In [7]:
#Limited perspectives: The group may become less diverse in terms of expertise, 
# which could limit the range of perspectives and ideas brought to bear on scientific problems. This could result in a narrower understanding of issues, and may make it more difficult to identify and solve problems.

#Reduced innovation: When there is a lack of diversity in a group of scientists, there is also a risk of stagnation and reduced innovation. Without new and varied ideas, the group may struggle to come up with creative solutions to problems or develop new approaches to research.

#Less collaboration: As the group becomes more insular, they may become less open to collaboration with researchers outside their immediate circle. This can limit opportunities for interdisciplinary work and the sharing of knowledge across different fields, which could ultimately slow down scientific progress.

pub2country['nCountries'] = pub2country.groupby(['PublicationId'])['Country'].transform('nunique')
pub2country

,PublicationId,Country,nCountries
0,3.147478e+09,BR,1
1,4.205869e+09,FR,1
2,4.225966e+09,BE,1
3,2.471332e+09,CN,1
4,2.367226e+09,CN,1
...,...,...,...
76593135,3.022242e+09,GB,1
76593136,2.396241e+09,CN,1
76593137,2.358301e+09,CN,1
76593138,2.953066e+09,US,1


In [8]:
pub2country['IsNational'] = pub2country['nCountries'] == 1
#national_rate = pub2country.groupby(['Country', 'Year'], as_index=False)['IsNational'].apply(lambda x: x.values.sum()/x.shape[0])
#national_rate.rename(columns = {'IsNational':'FractionNationalAuthors'}, inplace=True)

#national_rate.to_csv(os.path.join(path2countrydata, "oa_national_authorship_rate.csv.gz"), 
#                                 compression='gzip', mode='w', header=True, index=False)



In [9]:

cocountry = pub2country[~pub2country['IsNational']].reset_index(drop=True)
country2int = {c:i for i, c in enumerate(np.sort(cocountry['Country'].unique()))}
with open(os.path.join(path2countrydata, "oa_cocountry_collaboration_network_country2int.json"), 'w') as outfile:
    outfile.write(json.dumps(country2int))

In [35]:
cocountry = pub2country[~pub2country['IsNational']].reset_index(drop=True)
#years = np.sort(cocountry['Year'].unique())
years = np.arange(1970, 2023)
Nyears = years.shape[0]

country2int = {c:i for i, c in enumerate(np.sort(cocountry['Country'].unique()))}
int2country = {i:c for i, c in country2int.items()}

cocountry['Country'] = cocountry['Country'].map(country2int)
Ncountry = cocountry['Country'].nunique()

pub2int = {pid:i for i, pid in enumerate(np.sort(cocountry['PublicationId'].unique()))}
Npubs = cocountry['PublicationId'].nunique()
cocountry['PublicationId'] = cocountry['PublicationId'].map(country2int)

cocountry_net = np.zeros(shape=(Nyears,Ncountry,Ncountry))
for y in years:
    bipartite_adj = pyscisci.dataframe2bipartite(cocountry.loc[cocountry['Year'] == y], 
                                        'Country', 'PublicationId', (Ncountry, Npubs) )
    
    adj_mat = pyscisci.project_bipartite_mat(bipartite_adj, project_to = 'row')

    adj_mat.setdiag(0)
    adj_mat.eliminate_zeros()
    
    cocountry_net[y-1970] = adj_mat.todense()
    print(y)
    
np.save(os.path.join(path2countrydata, "oa_cocountry_collaboration_network.npy"), arr = cocountry_net)

1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


TypeError: _save_dispatcher() missing 1 required positional argument: 'arr'

# Diversity of countries publishing in journals

In [19]:
pub = pub.merge(pub2country, how='left', on='PublicationId')
pub

,PublicationId,Year,DocType,JournalId,Country
0,3147477878,2021.0,journal-article,4.210191e+09,BR
1,3092696942,2020.0,journal-article,9.868503e+07,NaN
2,2996677922,2019.0,journal-article,4.210208e+09,NaN
3,3128509038,2019.0,journal-article,4.210212e+09,NaN
4,4205868985,2019.0,proceedings-article,1.062967e+08,FR
...,...,...,...,...,...
140838956,2358301002,2009.0,journal-article,2.764507e+09,CN
140838957,2953065950,2020.0,journal-article,4.210235e+09,US
140838958,1799263988,2013.0,journal-article,2.764706e+09,NaN
140838959,2166352754,2006.0,journal-article,2.751447e+09,TH


In [21]:
countrydiversity = pub.dropna(subset=['JournalId', 'Country', 'Year']).groupby(['JournalId', 'Year'], as_index=False)['Country'].apply(pyscisci.simpson)
countrydiversity.rename(columns={'Country':'CountryDiversity'}, inplace=True)
countrydiversity

,JournalId,Year,CountryDiversity
0,6.166100e+04,1992.0,1.000000
1,6.166100e+04,1993.0,0.689877
2,6.166100e+04,1994.0,0.751634
3,6.166100e+04,1995.0,0.611852
4,6.166100e+04,1996.0,0.532407
...,...,...,...
1062569,4.306541e+09,2016.0,1.000000
1062570,4.306549e+09,2001.0,1.000000
1062571,4.306551e+09,1994.0,1.000000
1062572,4.306555e+09,1975.0,1.000000


In [22]:
countrydiversity.to_csv(os.path.join(path2countrydata, "oa_journal_diversity.csv.gz"), 
                                 compression='gzip', mode='w', header=True, index=False)

In [55]:
pubweight = pub.groupby(['Country', 'Year', 'JournalId'], as_index=False)['PublicationId'].nunique()
pubweight = pubweight.merge(countrydiversity, how='left', on=['JournalId', 'Year'])
pubweight

,Country,Year,JournalId,PublicationId,CountryDiversity
0,AD,1990.0,1.043534e+09,1,0.329370
1,AD,1993.0,5.287739e+07,1,0.079967
2,AD,1993.0,9.208579e+07,1,0.268879
3,AD,1998.0,4.306421e+09,1,0.103190
4,AD,2003.0,9.906897e+07,1,0.609204
...,...,...,...,...,...
10203976,ZW,2022.0,4.306403e+09,1,0.398645
10203977,ZW,2023.0,8.501616e+07,1,0.375000
10203978,ZW,2023.0,1.852973e+08,1,0.189014
10203979,ZW,2023.0,4.210202e+09,1,0.283951


In [59]:
def weighted_sum(x):
    return np.dot(x['PublicationId'].values, x['CountryDiversity'].values)/x['PublicationId'].sum()

country_journal_diversity = pubweight.groupby(['Country', 'Year'], as_index=False).apply(weighted_sum)
country_journal_diversity.rename(columns={None:'AveJournalDiversity'}, inplace=True)
country_journal_diversity

,Country,Year,AveJournalDiversity
0,AD,1990.0,0.329370
1,AD,1993.0,0.174423
2,AD,1998.0,0.103190
3,AD,2003.0,0.609204
4,AD,2006.0,0.468153
...,...,...,...
16949,ZW,2019.0,0.193901
16950,ZW,2020.0,0.218544
16951,ZW,2021.0,0.203225
16952,ZW,2022.0,0.200901


In [60]:
country_journal_diversity.to_csv(os.path.join(path2countrydata, "oa_country_journal_diversity.csv.gz"), 
                                 compression='gzip', mode='w', header=True, index=False)

In [67]:
pyscisci.simpson(['a', 'b', 'c'])

0.3333333333333333

# Topic Diversity

In [4]:
pub2country = pd.read_csv(os.path.join(path2countrydata, "oa_pub2country.csv.gz"))

In [5]:
focus_pubs = np.sort(pub2country['PublicationId'].unique())
pub2field = myoa.load_pub2field(columns=['PublicationId', 'FieldId'], filter_dict={'PublicationId':focus_pubs})
pub2field.shape

Loading Fields:   0%|          | 0/513 [00:00<?, ?it/s]

(877352959, 2)

In [6]:
fieldinfo = myoa.fieldinfo
fieldinfo

,FieldId,FieldName,WikiData,FieldLevel,NumberPublications,NumberCitations
0,150631331,Shell script,https://www.wikidata.org/wiki/Q959549,4,0,0
1,196351127,Strategic defence,https://www.wikidata.org/wiki/Q7621831,3,522,747
2,28838541,Flanging,https://www.wikidata.org/wiki/Q1323480,2,864,3198
3,2778416019,Natural person,https://www.wikidata.org/wiki/Q154954,2,664,230
4,206613426,Traditional economy,https://www.wikidata.org/wiki/Q2994466,2,570,871
...,...,...,...,...,...,...
65068,163439950,Data,https://www.wikidata.org/wiki/Q494756,2,86,88
65069,8324905,Uniform Code of Military Justice,https://www.wikidata.org/wiki/Q1424958,5,149,47
65070,9335704,AP United States Government and Politics,https://www.wikidata.org/wiki/Q4653504,4,40,300
65071,70784383,Nuclear reactor safety systems,https://www.wikidata.org/wiki/Q521918,3,118,42


In [11]:
lvl2 = np.sort(fieldinfo[fieldinfo['FieldLevel'] == 2]['FieldId'].values)

pubtopics2 = pub2field[pyscisci.isin_sorted(pub2field['FieldId'].values, lvl2)].merge(pub2country, how='left', on='PublicationId')
pubtopics2['Year'] = pubtopics2['PublicationId'].map(pub2year)

global_topic_div = pubtopics2.groupby(['Year'], as_index=False)['FieldId'].apply(pyscisci.simpson)
global_topic_div.rename(columns={'FieldId':'TopicDiversity'}, inplace=True)



In [16]:
global_topic_div.to_csv(os.path.join(path2countrydata, "oa_global_topic2_diversity.csv.gz"),
                        compression='gzip', mode='w', header=True, index=False)

In [14]:
pubtopics2_div = pubtopics2.groupby(['Country', 'Year'], as_index=False)['FieldId'].apply(pyscisci.simpson)
pubtopics2_div.rename(columns={'FieldId':'TopicDiversity'}, inplace=True)

In [15]:
pubtopics2_div.to_csv(os.path.join(path2countrydata, "oa_country_topic2_diversity.csv.gz"),
                        compression='gzip', mode='w', header=True, index=False)

In [10]:
lvl1= np.sort(fieldinfo[fieldinfo['FieldLevel'] == 1]['FieldId'].values)

pubtopics1 = pub2field[pyscisci.isin_sorted(pub2field['FieldId'].values, lvl1)].merge(pub2country, how='left', on='PublicationId')
pubtopics1['Year'] = pubtopics1['PublicationId'].map(pub2year)
print(pubtopics1.shape)

pubtopics1_div = pubtopics1.groupby(['Country', 'Year'], as_index=False)['FieldId'].apply(pyscisci.simpson)
pubtopics1_div.rename(columns={'FieldId':'TopicDiversity'}, inplace=True)

pubtopics1_div.to_csv(os.path.join(path2countrydata, "oa_country_topic1_diversity.csv.gz"),
                        compression='gzip', mode='w', header=True, index=False)
print(pubtopics1_div.shape)


(322054451, 4)
(16669, 3)


# Top Journals

In [ ]:
pub2country = pd.read_csv(os.path.join(path2countrydata, "oa_pub2country.csv.gz"))
c5 = pd.read_csv(os.path.join(path2countrydata, "oa_c5.csv.gz"))

In [ ]:
pub = myoa.load_publications(columns =['PublicationId', 'Year', 'DocType', 'JournalId'])
pub = pub.merge(pub2country, how='left', on='PublicationId')
pub = pub.merge(c5[['CitedPublicationId', 'C5']].drop_duplicates(), how='left', 
                left_on='PublicationId', right_on='CitedPublicationId')

pub

In [ ]:
kept_doctypes = np.sort(['article'])
pub.dropna(inplace=True)
print(pub.shape)
pub = pub[pyscisci.isin_sorted(pub['DocType'].values, kept_doctypes)].reset_index(drop=True)
print(pub.shape)
pub = pub[pub['Year'] >= 1900].reset_index(drop=True)
print(pub.shape)

In [ ]:
pub['logc5'] = np.log(pub['C5'])


In [ ]:
fieldinfo = myoa.fieldinfo
lvl1 = np.sort(fieldinfo[fieldinfo['FieldLevel'] == 1]['FieldId'].unique())
pub2field = myoa.load_pub2field(columns=['PublicationId', 'FieldId'], filter_dict={'FieldId':lvl1})

pub = pub.merge(pub2field, how='left', on='PublicationId').dropna()
pub

In [15]:
meanlogcite = pub.groupby(['Year', 'FieldId', 'JournalId'], as_index=False)['logc5'].mean()
meanlogcite

,Year,FieldId,JournalId,logc5
0,1900.0,2524010,1.056378e+08,0.231049
1,1900.0,2524010,4.210229e+09,0.000000
2,1900.0,10138342,4.210229e+09,0.000000
3,1900.0,11171543,4.210203e+09,0.000000
4,1900.0,13965031,1.056378e+08,0.000000
...,...,...,...,...
19170126,2017.0,2522767166,4.306531e+09,0.693147
19170127,2017.0,2522767166,4.306531e+09,2.079442
19170128,2017.0,2522767166,4.306531e+09,1.151293
19170129,2017.0,2522767166,4.306531e+09,0.000000


In [16]:
meanlogcite.to_csv(os.path.join(path2countrydata, 'oa_journal_field_meanlogc5.csv.gz'), index=False, 
                   mode='w', header=True, compression='gzip')

In [17]:
meanlogcite17 = meanlogcite[meanlogcite['Year'] == 2017].rese
njournals = meanlogcite17.groupby('FieldId')['JournalId'].nunique()

In [22]:
meanlogcite17['JournalRank'] = meanlogcite17.groupby('FieldId')['logc5'].transform(lambda a: pyscisci.rank_array(a, ascending=False, normed=False))

meanlogcite17

/var/folders/69/w28tw6xx26nd7x4yv_sh2mgm0000gp/T/ipykernel_24179/2185435163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meanlogcite17['JournalRank'] = meanlogcite17.groupby('FieldId')['logc5'].transform(lambda a: pyscisci.rank_array(a, ascending=False, normed=False))


,Year,FieldId,JournalId,logc5,JournalRank
18025090,2017.0,40700,2.677290e+05,2.663329,270
18025091,2017.0,40700,3.232370e+05,2.772589,226
18025092,2017.0,40700,6.306940e+05,1.329661,919
18025093,2017.0,40700,6.789650e+05,0.577623,1386
18025094,2017.0,40700,8.153010e+05,2.993383,171
...,...,...,...,...,...
19170126,2017.0,2522767166,4.306531e+09,0.693147,3497
19170127,2017.0,2522767166,4.306531e+09,2.079442,1830
19170128,2017.0,2522767166,4.306531e+09,1.151293,2934
19170129,2017.0,2522767166,4.306531e+09,0.000000,4251


In [26]:
print(meanlogcite17['FieldId'].nunique()*50)
top_journals = np.sort(meanlogcite17[meanlogcite17['JournalRank'] < 50]['JournalId'].unique())

14200


In [28]:
meanlogcite['JournalRank'] = meanlogcite.groupby(['Year', 'FieldId'])['logc5'].transform(lambda a: pyscisci.rank_array(a, ascending=False, normed=False))

meanlogcite

,Year,FieldId,JournalId,logc5,JournalRank
0,1900.0,2524010,1.056378e+08,0.231049,0
1,1900.0,2524010,4.210229e+09,0.000000,1
2,1900.0,10138342,4.210229e+09,0.000000,0
3,1900.0,11171543,4.210203e+09,0.000000,0
4,1900.0,13965031,1.056378e+08,0.000000,1
...,...,...,...,...,...
19170126,2017.0,2522767166,4.306531e+09,0.693147,3497
19170127,2017.0,2522767166,4.306531e+09,2.079442,1830
19170128,2017.0,2522767166,4.306531e+09,1.151293,2934
19170129,2017.0,2522767166,4.306531e+09,0.000000,4251


In [29]:
meanlogcite = meanlogcite[meanlogcite['Year'] >= 1990]
meanlogcite

,Year,FieldId,JournalId,logc5,JournalRank
2581707,1990.0,40700,1.234856e+06,1.074269,84
2581708,1990.0,40700,1.558358e+06,0.888939,98
2581709,1990.0,40700,2.228914e+06,0.693147,119
2581710,1990.0,40700,2.876017e+06,0.000000,171
2581711,1990.0,40700,3.416249e+06,0.541610,154
...,...,...,...,...,...
19170126,2017.0,2522767166,4.306531e+09,0.693147,3497
19170127,2017.0,2522767166,4.306531e+09,2.079442,1830
19170128,2017.0,2522767166,4.306531e+09,1.151293,2934
19170129,2017.0,2522767166,4.306531e+09,0.000000,4251


In [30]:
yearly_top_journals = meanlogcite[meanlogcite['JournalRank'] < 50]
yearly_top_journals

,Year,FieldId,JournalId,logc5,JournalRank
2581713,1990.0,40700,3.880285e+06,2.498514,7
2581716,1990.0,40700,5.353659e+06,1.945910,27
2581718,1990.0,40700,5.984737e+06,1.945910,25
2581721,1990.0,40700,9.731383e+06,1.519198,45
2581731,1990.0,40700,2.607397e+07,1.609438,42
...,...,...,...,...,...
19168790,2017.0,2522767166,4.210178e+09,5.293305,39
19168830,2017.0,2522767166,4.210182e+09,5.641907,19
19168910,2017.0,2522767166,4.210190e+09,6.711740,1
19169237,2017.0,2522767166,4.210230e+09,6.259581,4


In [31]:
yearly_top_journals.to_csv('oa_yearly_top_journals.csv.gz', index=False, 
                   mode='w', header=True, compression='gzip')

In [33]:
yearly_top_journals = yearly_top_journals[['Year', 'JournalId']].drop_duplicates()
yearly_top_journals['TopJournal'] = 1


In [32]:
country_pub = np.sort(pub2country['PublicationId'].unique())
pub2 = myoa.load_publications(columns =['PublicationId', 'Year', 'JournalId'], filter_dict = {'PublicationId':country_pub})
pub2.shape

Loading Publications:   0%|          | 0/513 [00:00<?, ?it/s]

/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Colu

/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Colu

/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fname, compression='gzip')
/Users/hgt6rn/Documents/GitHub/pyscisci/pyscisci/datasource/readwrite.py:155: DtypeWarning: Colu

(63898746, 3)

In [34]:
pub2 = pub2[pub2['Year']>=1990]
pub2 = pub2.merge(yearly_top_journals[['Year', 'JournalId', 'TopJournal']], how='left', on=['Year', 'JournalId'])
pub2.fillna(0, inplace=True)
pub2

,PublicationId,Year,JournalId,TopJournal
0,3147477878,2021.0,4.210191e+09,0.0
1,4205868985,2019.0,1.062967e+08,0.0
2,4225966262,2022.0,1.404626e+07,0.0
3,2471332289,2006.0,2.754768e+09,0.0
4,2367225558,2005.0,2.070659e+07,0.0
...,...,...,...,...
57483354,3022241891,2020.0,1.419533e+08,0.0
57483355,2396240571,2012.0,2.755648e+09,0.0
57483356,2358301002,2009.0,2.764507e+09,0.0
57483357,2953065950,2020.0,4.210235e+09,0.0


In [35]:
pub2 = pub2.merge(pub2country[['PublicationId', 'Country']], how='left', on='PublicationId')
pub2

,PublicationId,Year,JournalId,TopJournal,Country
0,3147477878,2021.0,4.210191e+09,0.0,BR
1,4205868985,2019.0,1.062967e+08,0.0,FR
2,4225966262,2022.0,1.404626e+07,0.0,BE
3,2471332289,2006.0,2.754768e+09,0.0,CN
4,2367225558,2005.0,2.070659e+07,0.0,CN
...,...,...,...,...,...
69754568,3022241891,2020.0,1.419533e+08,0.0,GB
69754569,2396240571,2012.0,2.755648e+09,0.0,CN
69754570,2358301002,2009.0,2.764507e+09,0.0,CN
69754571,2953065950,2020.0,4.210235e+09,0.0,US


In [36]:
print(pub2.shape)
pub2.drop_duplicates(subset=['PublicationId', 'Year', 'Country'], inplace=True)
print(pub2.shape)

(69754573, 5)
(69754566, 5)


In [37]:
totalprod = pub2.groupby(['Year', 'Country'], as_index=False)['PublicationId'].nunique().rename(columns={'PublicationId':'NumPub'})
numtop = pub2.groupby(['Year', 'Country'], as_index=False)['TopJournal'].sum()

countryprod = totalprod.merge(numtop, on=['Year', 'Country'])
countryprod

,Year,Country,NumPub,TopJournal
0,1990.0,AD,1,0.0
1,1990.0,AE,109,73.0
2,1990.0,AF,2,1.0
3,1990.0,AL,34,10.0
4,1990.0,AM,212,96.0
...,...,...,...,...
7222,2121.0,US,1,0.0
7223,2121.0,ZA,10,0.0
7224,2200.0,CY,1,0.0
7225,2202.0,FR,1,0.0


In [38]:
countryprod['FracTop'] = countryprod['TopJournal'] / countryprod['NumPub'] 


In [39]:
countryprod.to_csv(os.path.join(path2countrydata, "oa_countrytopproductivity.csv.gz"), compression='gzip', mode='w', header=True, index=False)


In [40]:
oasize = pub2.drop_duplicates(subset=['PublicationId']).groupby(['Year'], as_index=False)['PublicationId'].nunique().rename(columns={'PublicationId':'NumPub'})
numtop = pub2.drop_duplicates(subset=['PublicationId']).groupby(['Year'], as_index=False)['TopJournal'].sum()

oasize = oasize.merge(numtop, how='left', on='Year')
oasize

,Year,NumPub,TopJournal
0,1990.0,534873,339137.0
1,1991.0,574290,354427.0
2,1992.0,598442,366369.0
3,1993.0,626750,385591.0
4,1994.0,657471,391319.0
5,1995.0,681781,407563.0
6,1996.0,740613,431591.0
7,1997.0,773262,440247.0
8,1998.0,807327,453648.0
9,1999.0,843503,463412.0


In [41]:
oasize['FracTop'] = oasize['TopJournal'] / oasize['NumPub'] 
oasize

,Year,NumPub,TopJournal,FracTop
0,1990.0,534873,339137.0,0.634051
1,1991.0,574290,354427.0,0.617157
2,1992.0,598442,366369.0,0.612205
3,1993.0,626750,385591.0,0.615223
4,1994.0,657471,391319.0,0.595188
5,1995.0,681781,407563.0,0.597792
6,1996.0,740613,431591.0,0.582748
7,1997.0,773262,440247.0,0.569337
8,1998.0,807327,453648.0,0.561914
9,1999.0,843503,463412.0,0.549390


In [45]:
oasize = oasize[oasize['Year']<=2017].rename(columns={'NumPub':'OANumPub', 'TopJournal':'OATopJournal', 'FracTop':"OAFracTop"})
oasize

,Year,OANumPub,OATopJournal,OAFracTop
0,1990.0,534873,339137.0,0.634051
1,1991.0,574290,354427.0,0.617157
2,1992.0,598442,366369.0,0.612205
3,1993.0,626750,385591.0,0.615223
4,1994.0,657471,391319.0,0.595188
5,1995.0,681781,407563.0,0.597792
6,1996.0,740613,431591.0,0.582748
7,1997.0,773262,440247.0,0.569337
8,1998.0,807327,453648.0,0.561914
9,1999.0,843503,463412.0,0.549390


In [47]:
oasize.to_csv(os.path.join(path2countrydata, "oa_globaltopjournal.csv.gz"), compression='gzip', mode='w', header=True, index=False)

# Process publication abstracts for ideas

In [ ]:
pub = myoa.load_publications(columns=['PublicationId', 'Year', 'DocType', 'Title'],
                      dropna = ['PublicationId', 'Year', 'DocType', 'Title'])
pub2country = pd.read_csv(os.path.join(path2countrydata, "oa_pub2country.csv.gz"))

pub = pub.merge(pub2country, how='inner')

pub.shape

In [ ]:
idealist = pd.read_csv('data/Chengetal_idealist.csv.gz')
ideas = idealist['id'].unique()
print(ideas.shape)

In [ ]:
vectorizer = CountVectorizer(vocabulary = ideas, ngram_range=(1,6))

punct2remove = string.punctuation
punct2remove = punct2remove.replace(".", "").replace(",", "") # don't remove period, comma
pattern2remove = r"[{}]".format(punct2remove) # create the pattern

In [ ]:
if not os.path.exists(os.path.join(path2openalex, 'pub2idea')):
    os.mkdir(os.path.join(path2openalex, 'pub2idea'))

In [ ]:

pub2keyword = []

path2abstracts = os.path.join(path2openalex, 'pub2abstract')
abstract_files = [file for (dirpath, dirnames, filenames) in os.walk(path2abstracts) for file in filenames if '.gz' in file]

sfile = 0
for ifile in range(len(abstract_files)):
    
    pubinfo = pd.read_csv(os.path.join(path2openalex, 'publication', 'publication{}.csv.gz'.format(ifile)))
    pubinfo = pubinfo[['PublicationId', 'Title']]
    pubinfo = pubinfo.set_index('PublicationId')
    
    with gzip.open(os.path.join(path2openalex, 'pub2abstract', 'pub2abstract{}.jsonl.gz'.format(ifile))) as infile:
        
        for line in infile:
            publine = json.loads(line.decode('utf8'))
            for k, a in publine.items():
                if not a is None and len(a) > 0:
                    try:
                        fullstr = str(pubinfo.loc[int(k)]['Title']) + '. ' +  pyscisci.abstractindex2text(a)
                    except KeyError:
                        fullstr = pyscisci.abstractindex2text(a)
                    fullstr = fullstr.lower()

                    fullstr = re.sub(r'\d', '', fullstr)

                    fullstr = fullstr.translate(str.maketrans("", "", pattern2remove))
                    word_tokens_stem = " ".join([ps.stem(w) for w in word_tokenize(fullstr)])
                    #print(fullstr)

                    found_ideas = vectorizer.transform([word_tokens_stem])
                    r, c = found_ideas.nonzero()

                    for idx in c:
                        pub2keyword.append([int(k), ideas[idx]])
    print(ifile)
    if ifile % 50 == 0:
        pub2keyword2 = pd.DataFrame(pub2keyword, columns=['PublicationId', 'Idea'])
        pub2keyword2.to_csv(os.path.join(path2openalex, 'pub2idea', 'pub2idea{}.csv.gz'.format(sfile)),
                  mode='w', compression='gzip', header=True, index=False)
        sfile += 1
        pub2keyword = []
        
pub2keyword = pd.DataFrame(pub2keyword, columns=['PublicationId', 'Idea'])
pub2keyword.to_csv(os.path.join(path2openalex, 'pub2idea', 'pub2idea{}.csv.gz'.format(sfile)),
                  mode='w', compression='gzip', header=True, index=False)

pub2keyword